In [3]:
include("../src/HDC.jl")
include("../src/math.jl")
include("../src/experimental.jl")
using JLD

phalp_bow_esm = load("../data/phalp_bow_esm.jld")["embedded_bow_esm"]
phalp_bow_rand = load("../data/phalp_bow_rand.jld")["embedded_rand_bow"]
phalp_cnn_esm = load("../data/phalp_CNN_esm.jld")["embedded_CNN_esm"]
upi2doms = load("../data/phalp_upi2doms.jld")["upi2doms"]
up2type = load("../data/phalp_UP2type.jld")["UP2type"]
domacc2domname = load("../data/phalp_domacc2domname.jld")["domacc2domname"]
phalp_cnn_rand = load("../data/phalp_cnn_rand.jld")["embedded_rand_CNN"]


┌ Warning: type InlineStrings.String15 not present in workspace; reconstructing
└ @ JLD /home/mfat/.julia/packages/JLD/S6t6A/src/jld_types.jl:697
┌ Warning: type InlineStrings.String31 not present in workspace; reconstructing
└ @ JLD /home/mfat/.julia/packages/JLD/S6t6A/src/jld_types.jl:697


┌ Warning: type InlineStrings.String31 not present in workspace; reconstructing
└ @ JLD /home/mfat/.julia/packages/JLD/S6t6A/src/jld_types.jl:697


Dict{Any, Any} with 11549 entries:
  "UPI000178DD30" => Bool[0, 1, 0, 0, 0, 0, 0, 1, 1, 1  …  0, 0, 1, 1, 0, 0, 0,…
  "UPI0013EFDC93" => Bool[1, 1, 1, 0, 1, 0, 0, 0, 0, 0  …  0, 0, 0, 0, 1, 1, 0,…
  "UPI000172D062" => Bool[1, 0, 0, 0, 0, 1, 1, 0, 1, 1  …  0, 1, 1, 0, 1, 0, 0,…
  "UPI001463ABBB" => Bool[0, 0, 0, 0, 1, 0, 1, 1, 1, 0  …  1, 0, 1, 0, 1, 1, 1,…
  "UPI000232F56D" => Bool[0, 0, 1, 0, 0, 0, 1, 0, 1, 0  …  1, 0, 0, 1, 0, 0, 0,…
  "UPI0011625D30" => Bool[0, 0, 1, 0, 0, 1, 0, 0, 1, 1  …  0, 1, 0, 0, 1, 1, 1,…
  "UPI0009882324" => Bool[1, 1, 0, 1, 0, 0, 0, 0, 1, 0  …  1, 0, 0, 0, 1, 0, 1,…
  "UPI000CA1D611" => Bool[1, 0, 1, 0, 0, 0, 1, 0, 0, 0  …  1, 0, 0, 1, 0, 0, 0,…
  "UPI0006BC2F8A" => Bool[0, 1, 1, 1, 1, 1, 1, 0, 0, 0  …  0, 0, 1, 1, 1, 0, 0,…
  "UPI000BBF7878" => Bool[0, 1, 1, 1, 0, 1, 1, 1, 1, 1  …  1, 1, 0, 1, 1, 1, 1,…
  "UPI00138B2696" => Bool[0, 1, 1, 0, 0, 0, 0, 0, 0, 1  …  0, 0, 1, 1, 1, 1, 0,…
  "UPI00025D6AED" => Bool[1, 0, 0, 0, 0, 0, 0, 1, 0, 0  …  1, 1, 0, 1, 1, 

# Type classifier

In [4]:
function simp_trainer(seq_dict, feature_dict)
"""
input: dict(names, sequences) and dict(names, class)
"""

    inv_fd = Dict()
    for k in keys(feature_dict)
        if feature_dict[k] in keys(inv_fd)
            push!(inv_fd[feature_dict[k]],k)
        else
            inv_fd[feature_dict[k]] = [k]
        end
    end

    types = collect(keys(inv_fd))

    class_hdvs = Dict()
    for i in types
        names_intr = inv_fd[i]
        seq = [seq_dict[n] for n in names_intr]
        class_hdvs[i] = bitadd(hcat(seq)...)
    end

    return class_hdvs
end


simp_trainer (generic function with 1 method)

In [5]:
function simp_train2test(seq, feature)
    """
    input: dict(names, sequences) and dict(names, class)
    """
        class_hdvs = Dict()
        Threads.@threads for i in 1:length(feature)
            if feature[i] in keys(class_hdvs)
                class_hdvs[feature[i]] = bitadd(class_hdvs[feature[i]], seq[i])
            else
                class_hdvs[feature[i]] = seq[i]
            end
        end
    
        return class_hdvs
    end
    

simp_train2test (generic function with 1 method)

In [6]:
using MLDataPattern
function simp_test(seq_dict, feature_dict)
"""
input: dict(names, sequences) and dict(names, class), output simp_trainer
"""
    Keys = [key for (key, val) in feature_dict]
    key_seq = [seq_dict[i] for i in Keys]
    Values = [val for (key, val) in feature_dict]

    (X_train,y_train), (X_test,y_test) = stratifiedobs((key_seq, Values), p = 0.8)
    classes = collect(Set(y_test))
    n = length(classes)

    class_ind = Dict()
    Threads.@threads for i in 1:n
        class_ind[classes[i]] = i
    end

    confusion_matrix = zeros(n,n)

    class_hdv = simp_train2test(X_train, y_train)

    Threads.@threads for i in 1:length(X_test)
        for j in classes
            simil = 0
            new_simil = hamming(X_test[i], class_hdv[j])
            if new_simil > simil
                simil = new_simil
                pred = j
            end
        confusion_matrix[class_ind[y_test[i]], class_ind[j]] += 1
        end
    end
    return confusion_matrix
end

simp_test (generic function with 1 method)

In [7]:
simp_test(phalp_bow_rand, up2type)

2×2 Matrix{Float64}:
 1581.0  1594.0
  672.0   675.0

oof